In [1]:
import requests

In [2]:
from lxml.html import document_fromstring

In [3]:
data = requests.get("https://www.cia.gov/library/publications/resources/the-world-factbook/docs/rankorderguide.html").text

In [5]:
data = document_fromstring(data)

In [7]:
attrs = data.xpath("//*[@id='profileguide']/*/div/strong/a")

In [20]:
attrs[0].attrib['href'], attrs[0].text

('../fields/279rank.html', 'Area:')

In [26]:
url = "https://www.cia.gov/library/publications/resources/the-world-factbook/"+attrs[0].attrib['href'][3:]
url

'https://www.cia.gov/library/publications/resources/the-world-factbook/fields/279rank.html'

In [27]:
def to_lxml(url):
    return document_fromstring(requests.get(url).text)

In [28]:
def get_data(url):
    data = to_lxml(url)
    country = data.xpath("//*[@id='rankOrder']/tbody/*/td[2]/a/text()")
    value = data.xpath("//*[@id='rankOrder']/tbody/*/td[3]/text()")
    return [country, value]

In [40]:
all_data = []
for attr in attrs:
    field = attr.text
    url = "https://www.cia.gov/library/publications/resources/the-world-factbook/"+attr.attrib['href'][3:]
    all_data.append([field, get_data(url)])

In [45]:
region_data = to_lxml("https://www.cia.gov/library/publications/resources/the-world-factbook/fields/279rank.html")

In [47]:
regions = region_data.xpath("//*[@id='rankOrder']/tbody/tr/@class")
regions = [r[10:] for r in regions]

In [57]:
country_data = {}
country = all_data[0][1][0]

In [60]:
for i, c in enumerate(country):
    country_data[c] = [c, regions[i]]

In [78]:
data_s = {}
all_attr = ["Country", "Continent"]
for attr_data in all_data:
    attr, d = attr_data
    all_attr.append(attr[:-1])
    c, v = d
    data_s[attr] = {c[i]: v[i] for i in range(len(c))}

In [71]:
for c in country:
    for data in all_data:
        attr = data[0]
        value = ""
        if c in data_s[attr]:
            value = data_s[attr][c]
        country_data[c].append(value)

In [79]:
all_attr

['Country',
 'Continent',
 'Area',
 'Population',
 'Median age',
 'Population growth rate',
 'Birth rate',
 'Death rate',
 'Net migration rate',
 'Maternal mortality rate',
 'Infant mortality rate',
 'Life expectancy at birth',
 'Total fertility rate',
 'Health expenditures',
 'HIV/AIDS - adult prevalence rate',
 'HIV/AIDS - people living with HIV/AIDS',
 'HIV/AIDS - deaths',
 'Obesity - adult prevalence rate',
 'Children under the age of 5 years underweight',
 'Education expenditures',
 'Unemployment, youth ages 15-24',
 'GDP (purchasing power parity)',
 'GDP - real growth rate',
 'GDP - per capita (PPP)',
 'Gross national saving',
 'Industrial production growth rate',
 'Labor force',
 'Unemployment rate',
 'Distribution of family income - Gini index',
 'Taxes and other revenues',
 'Budget surplus (+) or deficit (-)',
 'Public debt',
 'Inflation rate (consumer prices)',
 'Central bank discount rate',
 'Commercial bank prime lending rate',
 'Stock of narrow money',
 'Stock of broad mon

In [80]:
import csv

with open("data.csv","w") as csvfile: 
    writer = csv.writer(csvfile)
    writer.writerow(all_attr)
    for c in country:
        writer.writerow(country_data[c])